In [ ]:
# Importar chave privada SSH
import json
from google.colab import files
uploaded = files.upload()

Saving key.pem to key.pem


In [ ]:
# Corrigir permissões da chave privada se necessário
!chmod 600 key.pem

In [ ]:
# Instalar o Ansible
!apt-get update && apt-get install -y software-properties-common
!apt-add-repository --yes --update ppa:ansible/ansible
!apt-get install -y ansible

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,321 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,664 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,533 kB]
Hit:13 https://ppa.launchpadcontent.

In [ ]:
# Checar a instalação
!ansible --version

ansible [core 2.17.9]
  config file = /etc/ansible/ansible.cfg
  configured module search path = ['/root/.ansible/plugins/modules', '/usr/share/ansible/plugins/modules']
  ansible python module location = /usr/lib/python3/dist-packages/ansible
  ansible collection location = /root/.ansible/collections:/usr/share/ansible/collections
  executable location = /usr/bin/ansible
  python version = 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0] (/usr/bin/python3)
  jinja version = 3.1.5
  libyaml = True


In [ ]:
%%writefile hosts

[dbservers]
34.72.238.18 ansible_ssh_user=danib ansible_ssh_private_key_file=key.pem ansible_ssh_common_args='-o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null'

Overwriting hosts


In [ ]:
# Tentativa MAL SUCEDIDA de instlar o MySQL
%%writefile db-playbook.yml
- hosts: all
  become: yes
  tasks:
    - name: Atualizar o repositório apt
      apt:
        update_cache: yes

    - name: Instalar dependências necessárias
      apt:
        name:
          - software-properties-common
          - gnupg
          - wget
        state: present

    - name: Baixar a chave GPG do MySQL
      shell: |
        wget -qO - https://repo.mysql.com/RPM-GPG-KEY-mysql-2022 |
        gpg --dearmor > /etc/apt/trusted.gpg.d/mysql.gpg

    - name: Adicionar repositório oficial do MySQL
      apt_repository:
        repo: "deb [arch=amd64] http://repo.mysql.com/apt/debian bullseye mysql-8.0"
        state: present

    - name: Atualizar o repositório apt novamente
      apt:
        update_cache: yes

    - name: Instalar o MySQL server
      apt:
        name: mysql-server
        state: present

    - name: Reiniciar o MySQL
      service:
        name: mysql
        state: restarted

    - name: Configurar senha de root do MySQL
      debconf:
        name: 'mysql-server'
        question: 'mysql-server/root_password'
        value: 'petAdot@2025'
        vtype: 'password'

    - name: Configurar senha de root do MySQL (confirmation)
      debconf:
        name: 'mysql-server'
        question: 'mysql-server/root_password_again'
        value: 'petAdot@2025'
        vtype: 'password'

    - name: Certificar que o MySQL está rodando
      service:
        name: mysql
        state: started
        enabled: yes

Overwriting db-playbook.yml


In [ ]:
# Instalação do Maria DB

%%writefile db-playbook.yml
- hosts: all
  become: yes
  tasks:
    - name: Atualizar o repositório apt
      apt:
        update_cache: yes

    - name: Instalar o MariaDB server
      apt:
        name: mariadb-server
        state: present

    - name: Reiniciar o MariaDB
      service:
        name: mariadb
        state: restarted

    - name: Configurar senha de root do MariaDB
      debconf:
        name: 'mariadb-server'
        question: 'mariadb-server/root_password'
        value: 'petAdot@2025'
        vtype: 'password'

    - name: Configurar senha de root do MariaDB (confirmation)
      debconf:
        name: 'mariadb-server'
        question: 'mariadb-server/root_password_again'
        value: 'petAdot@2025'
        vtype: 'password'

    - name: Certificar que o MariaDB está rodando
      service:
        name: mariadb
        state: started
        enabled: yes

Overwriting db-playbook.yml


In [ ]:
!ansible-playbook -i hosts db-playbook.yml --key-file key.pem -vv

/usr/lib/python3/dist-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
/usr/lib/python3/dist-packages/paramiko/transport.py:261: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
ansible-playbook [core 2.17.9]
  config file = /etc/ansible/ansible.cfg
  configured module search path = ['/root/.ansible/plugins/modules', '/usr/share/ansible/plugins/modules']
  ansible python module location = /usr/lib/python3/dist-p

In [ ]:
# Usado para tentar resolver a incompatibilidade do MySQl
%%writefile check_facts.yml
- hosts: all
  become: yes
  tasks:
    - name: Exibir distribuição e versão
      debug:
        msg: "Distribuição: {{ ansible_distribution }}, Versão: {{ ansible_distribution_version }}, Nome: {{ ansible_distribution_release }}"

Overwriting check_facts.yml


In [ ]:
!ansible-playbook -i hosts check_facts.yml

/usr/lib/python3/dist-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
/usr/lib/python3/dist-packages/paramiko/transport.py:261: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
[WARNING]: Platform linux on host 34.72.238.18 is using the discovered Py